# CLEAN COLAB NOTEBOOK FOR MD&A EXTRACTION FROM EDGAR 10-K FILINGS


# SECTION 1: SETUP


In [4]:
## Cell 1: Mount Google Drive

import os
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Drive remounted successfully")
else:
    print("❌ Drive still not accessible")

✅ Drive remounted successfully


In [5]:
## Cell 1.1: Mount Google Drive

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
## Cell 2: Navigate to Project Directory
import os
os.chdir('/content/drive/MyDrive/EDGAR_Project/edgar-crawler')
!pwd  # Verify location


/content/drive/MyDrive/EDGAR_Project/edgar-crawler


In [7]:
## Cell 3: Install Dependencies
print("📦 Installing dependencies...")

# Install compatible versions to avoid conflicts
!pip install -q 'dill<0.3.9' 'multiprocess<0.70.17'
!pip install -q pox ppft
!pip install -q --no-deps pathos
!pip install -q beautifulsoup4 lxml requests pandas tqdm click cssutils numpy

print("✅ All dependencies installed")


📦 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 7.2 MB/s eta 0:00:00
✅ All dependencies installed


# SECTION 2: REBUILD METADATA (ONE-TIME SETUP)


In [ ]:
## Cell 4: Download Metadata Rebuilder Script
!wget -q https://raw.githubusercontent.com/haowenluo/edgar-crawler/claude/fix-edgar-inventory-01VHYDnmaNQtFpWqjL8B6Bwq/rebuild_metadata_colab.py
print("✅ Script downloaded")


In [ ]:
## Cell 5: Rebuild Metadata from All Files on Disk
from rebuild_metadata_colab import rebuild_for_colab
import pandas as pd

# Fast mode: extracts CIK, Type, year, accession_number from filenames (~5-10 min)
rebuild_for_colab(filing_types=['10-K'], fast_mode=True, dry_run=False)

# Add required columns for extraction
metadata = pd.read_csv('datasets/FILINGS_METADATA.csv')

# Add all missing columns
required_columns = {
    'Company': lambda: 'Company_' + metadata['CIK'].astype(str),
    'Date': lambda: metadata['year'].astype(str) + '-01-01',
    'filing_date': lambda: metadata['year'].astype(str) + '-01-01',
    'Period of Report': lambda: metadata['year'].astype(str) + '-12-31',
    'SIC': lambda: 'Unknown',
    'State of Inc': lambda: 'Unknown',
    'State location': lambda: 'Unknown',
    'Fiscal Year End': lambda: '1231',
    'html_index': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
    'complete_text_file_link': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
    'htm_file_link': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
}

for col_name, col_value_func in required_columns.items():
    if col_name not in metadata.columns:
        metadata[col_name] = col_value_func()

metadata.to_csv('datasets/FILINGS_METADATA.csv', index=False)

print(f"\n✅ Metadata complete: {len(metadata):,} filings ready for extraction")
print(f"   Unique companies: {metadata['CIK'].nunique():,}")
print(f"   Years: {sorted(metadata['year'].unique())}")


# SECTION 3: CONFIGURE EXTRACTION


In [ ]:
## Cell 6: Update Config to Use Rebuilt Metadata
import json

config_path = 'config.json'

with open(config_path, 'r') as f:
    config = json.load(f)

config['extract_items']['filings_metadata_file'] = 'FILINGS_METADATA.csv'
config['extract_items']['filing_types'] = ['10-K']

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Config updated for MD&A extraction")


In [ ]:
## Cell 7: Apply Fix for Year-Based Subdirectories
file_path = 'extract_items.py'

with open(file_path, 'r') as f:
    content = f.read()

# Only apply if not already patched
if 'search in year subdirectories' not in content:
    old_code = '''        absolute_filename = os.path.join(
            self.raw_files_folder, filing_metadata["Type"], filing_metadata["filename"]
        )

        # Read the content of the file
        with open(absolute_filename, "r", errors="backslashreplace") as file:
            content = file.read()'''

    new_code = '''        # First try direct path (original behavior)
        absolute_filename = os.path.join(
            self.raw_files_folder, filing_metadata["Type"], filing_metadata["filename"]
        )

        # If file doesn't exist, search in year subdirectories
        if not os.path.exists(absolute_filename):
            type_dir = os.path.join(self.raw_files_folder, filing_metadata["Type"])
            found = False
            for root, dirs, files in os.walk(type_dir):
                if filing_metadata["filename"] in files:
                    absolute_filename = os.path.join(root, filing_metadata["filename"])
                    found = True
                    break

            if not found:
                raise FileNotFoundError(f"Could not find {filing_metadata['filename']} in {type_dir}")

        # Read the content of the file
        with open(absolute_filename, "r", errors="backslashreplace") as file:
            content = file.read()'''

    content = content.replace(old_code, new_code)

    with open(file_path, 'w') as f:
        f.write(content)

    print("✅ extract_items.py patched for nested directories")
else:
    print("✅ extract_items.py already patched")


# SECTION 4: EXTRACT MD&A


In [ ]:
## Cell 8: Run MD&A Extraction
!python flexible_extractor.py --config extraction_configs/mda_only.json


# SECTION 5: CHECK PROGRESS


In [ ]:
## Cell 9: Check Extraction Progress
import os
import json

extracted_dir = 'datasets/EXTRACTED_FILINGS/10-K'

if os.path.exists(extracted_dir):
    all_files = [f for f in os.listdir(extracted_dir) if f.endswith('.json')]

    # Get expected total from metadata
    metadata = pd.read_csv('datasets/FILINGS_METADATA.csv')
    expected = len(metadata[metadata['Type'] == '10-K'])

    print(f"📊 Extraction Progress:")
    print(f"   Extracted: {len(all_files):,} files")
    print(f"   Expected: {expected:,} files")
    print(f"   Progress: {len(all_files)/expected*100:.1f}%")
    print(f"   Remaining: {expected - len(all_files):,} files")

    # Check sample files for MD&A content
    print(f"\n📋 Sample Quality Check:")
    for fname in all_files[:3]:
        fpath = os.path.join(extracted_dir, fname)
        with open(fpath, 'r') as f:
            data = json.load(f)
            has_mda = 'item_7' in data and len(data.get('item_7', '')) > 100
            mda_len = len(data.get('item_7', ''))
            print(f"   {fname}: {'✅' if has_mda else '❌'} MD&A ({mda_len:,} chars)")
else:
    print("❌ No extraction directory found")


# SECTION 6: CREATE ANALYSIS FILES


In [ ]:
## Cell 10: Create Metadata CSV and Parquet for Analysis
import pandas as pd
import json
import os
from tqdm import tqdm

print("📊 Creating analysis files...")

extracted_dir = 'datasets/EXTRACTED_FILINGS/10-K'
metadata_records = []
full_data_records = []

# Process all extracted JSON files
json_files = [f for f in os.listdir(extracted_dir) if f.endswith('.json')]

for filename in tqdm(json_files, desc="Processing"):
    filepath = os.path.join(extracted_dir, filename)

    try:
        with open(filepath, 'r') as f:
            filing = json.load(f)

        # Metadata (lightweight)
        metadata_records.append({
            'filename': filename,
            'cik': filing.get('cik', ''),
            'company': filing.get('company', ''),
            'filing_date': filing.get('filing_date', ''),
            'period_of_report': filing.get('period_of_report', ''),
            'year': filing.get('period_of_report', '')[:4] if filing.get('period_of_report', '') else '',
            'has_mda': 'item_7' in filing and len(filing.get('item_7', '')) > 0,
            'mda_length': len(filing.get('item_7', '')),
            'json_path': filepath
        })

        # Full data (with MD&A text)
        if 'item_7' in filing and filing['item_7']:
            full_data_records.append({
                'cik': filing.get('cik', ''),
                'company': filing.get('company', ''),
                'filing_date': filing.get('filing_date', ''),
                'period_of_report': filing.get('period_of_report', ''),
                'year': filing.get('period_of_report', '')[:4] if filing.get('period_of_report', '') else '',
                'mda_text': filing.get('item_7', '')
            })
    except Exception as e:
        print(f"⚠️  Error: {filename} - {e}")

# Save files
df_meta = pd.DataFrame(metadata_records)
df_full = pd.DataFrame(full_data_records)

meta_path = '/content/drive/MyDrive/EDGAR_Project/mda_metadata.csv'
parquet_path = '/content/drive/MyDrive/EDGAR_Project/mda_full.parquet'

df_meta.to_csv(meta_path, index=False)
df_full.to_parquet(parquet_path, compression='gzip', index=False)

print(f"\n✅ Files created:")
print(f"   📄 Metadata CSV: {len(df_meta):,} records ({os.path.getsize(meta_path)/1024:.1f} KB)")
print(f"   📦 Parquet: {len(df_full):,} records ({os.path.getsize(parquet_path)/(1024**2):.1f} MB)")
print(f"\n📊 Statistics:")
print(f"   Years covered: {sorted(df_full['year'].unique())}")
print(f"   MD&A length: {df_full['mda_text'].str.len().describe()}")
print(f"\n🎉 Ready for analysis!")


# OPTIONAL: RESUME EXTRACTION AFTER INTERRUPTION


In [ ]:
## Cell 11: Resume Extraction (Run this if Colab disconnects)
# Reinstall dependencies
!pip install -q 'dill<0.3.9' 'multiprocess<0.70.17' pox ppft
!pip install -q --no-deps pathos
!pip install -q beautifulsoup4 lxml requests pandas tqdm click cssutils numpy

# Resume extraction
!python flexible_extractor.py --config extraction_configs/mda_only.json
